In [3]:
import json #A
from urllib import request, parse

def link_to_title(link): #B
    return link["title"]

def clean_if_key(page,key): #C
    if key in page.keys():
        return map(link_to_title,page[key])
    else: return []

def get_wiki_links(pageTitle): #D
    safe_title = parse.quote(pageTitle) #E
    url = "https://en.wikipedia.org/w/api.php?action=query&prop=links|linkshere&pllimit=500&lhlimit=500&titles={}&format=json&formatversion=2".format(safe_title)
    page = request.urlopen(url).read() #G
    j = json.loads(page) #H
    jpage = j['query']['pages'][0]
    inbound = clean_if_key(jpage,"links") #I
    outbound = clean_if_key(jpage,"linkshere")
    return {"title": pageTitle, #J
            "in-links":list(inbound),
            "out-links":list(outbound)}

In [4]:
def flatten_network(page):
    return page["in-links"]+page["out-links"]

In [5]:
def page_to_edges(page):
    a = [(page['title'],p) for p in page['out-links']]
    b = [(p,page['title']) for p in page['in-links']]
    return a+b

In [6]:
from multiprocessing import Pool

root = get_wiki_links("Parallel_computing")
initial_network = flatten_network(root)
with Pool() as P:
    all_pages = P.map(get_wiki_links, initial_network)
    edges = P.map(page_to_edges, all_pages)

In [7]:
from itertools import chain
edges = chain.from_iterable(edges)

In [ ]:
import networkx as nx
G = nx.DiGraph()
for e in edges:
    G.add_edge(*e)
    nx.readwrite.gexf.write_gexf(G,"./MyGraph.gexf")